# Learning objectives:
By the end, you will be able to...
- Understand how to create new columns based on existing data
- Use `.apply()` and `lambda` functions for row-wise operations
- Computed sum scores and derived epidemiological variables (e.g., BMI, PHQ2)

In [1]:
import pandas as pd

# Sample data
df = pd.DataFrame({
    'height_cm': [170, 160, 180],
    'weight_kg': [70, 60, 90],
    'symptom1': [1, 0, 1],
    'symptom2': [1, 1, 0],
    'symptom3': [0, 0, 1]
})
df

height_cm  weight_kg  symptom1  symptom2  symptom3
0        170         70         1         1         0
1        160         60         0         1         0
2        180         90         1         0         1

## A simple derived variable

In [3]:
# Convert height and weight to BMI
df['height_m'] = df['height_cm'] / 100
df['BMI'] = df['weight_kg'] / (df['height_m'] ** 2)
df

height_cm  weight_kg  symptom1  symptom2  symptom3  height_m        BMI
0        170         70         1         1         0       1.7  24.221453
1        160         60         0         1         0       1.6  23.437500
2        180         90         1         0         1       1.8  27.777778

# Using `.apply()` with a `lambda` function for row-wise calculations

In [5]:
# We've already seen many functions in Python: len() mean() pd.DataFrame()
# A lambda function is a new function that we create on-the-fly

# Apply row-wise lambda function to create a flag for overweight (BMI > 25)
df['overweight_flag'] = df['BMI'].apply(lambda x: 1 if x > 25 else 0)
df[['BMI','overweight_flag']]

BMI  overweight_flag
0  24.221453                0
1  23.437500                0
2  27.777778                1

# Creating sum scores

In [6]:
# Let's look at the sum of symptoms in this df
df['symptom_total'] = df[['symptom1','symptom2','symptom3']].sum(axis=1)
df

height_cm  weight_kg  symptom1  symptom2  symptom3  height_m        BMI  \
0        170         70         1         1         0       1.7  24.221453   
1        160         60         0         1         0       1.6  23.437500   
2        180         90         1         0         1       1.8  27.777778   

   overweight_flag  symptom_total  
0                0              2  
1                0              1  
2                1              2

# Applying this to the Oregon dataset

In [7]:
# Make Google Drive available to the script
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# Let's load the Oregon Health Insurance Experiment dataset
filename = 'drive/MyDrive/Colab Notebooks/Intro to Python for Epidemiologists/Data/OHIE_12m.csv'
OHIE = pd.read_csv(filename)
OHIE.head()

person_id  household_id     treatment  \
0      64350        164350  Not selected   
1      55655        155655  Not selected   
2      20087        128134      Selected   
3      70998        170998  Not selected   
4       8839        108839      Selected   

                               draw_treat    draw_lottery  \
0                                     NaN  Lottery Draw 6   
1                                     NaN  Lottery Draw 7   
2  Draw 6: selected in lottery 07/01/2008  Lottery Draw 6   
3                                     NaN  Lottery Draw 7   
4  Draw 8: selected in lottery 09/02/2008  Lottery Draw 8   

                            applied_app approved_app dt_notify_lottery  \
0                                   NaN          NaN        2008-07-14   
1                                   NaN          NaN        2008-08-12   
2       Submitted an Application to OHP           No        2008-07-14   
3                                   NaN          NaN        2008-08-12   
4  Did NOT submit an application to OHP           No        2008-09-11   

  dt_retro_coverage  birthyear_list  ... live_partner_12m live_parents_12m  \
0        2008-08-08            1974  ...               No              Yes   
1        2008-09-08            1987  ...              Yes               No   
2        2008-08-08            1963  ...               No               No   
3        2008-09-08            1954  ...              Yes               No   
4        2008-10-08            1964  ...               No               No   

   live_friends_12m live_relatives_12m live_other_12m hhsize_12m PHQ2_1  \
0                No                 No             No        2.0    3.0   
1                No                 No             No        2.0    1.0   
2                No                Yes             No        7.0    0.0   
3                No                 No             No        2.0    3.0   
4               Yes                 No             No        4.0    2.0   

  PHQ2_2 PHQ2_sum PHQ2_cutoff  
0    3.0      6.0        True  
1    1.0      2.0       False  
2    1.0      1.0       False  
3    2.0      5.0        True  
4    2.0      4.0        True  

[5 rows x 44 columns]

In [9]:
OHIE[['dep_interest_12m','dep_sad_12m']].head()

dep_interest_12m              dep_sad_12m
0         nearly every day         nearly every day
1             several days             several days
2             several days               not at all
3  more than half the days         nearly every day
4  more than half the days  more than half the days

In [10]:
phq_scoring = {'not at all': 0,
               'several days': 1,
               'more than half the days': 2,
               'nearly every day': 3}
phq_scoring

{'not at all': 0,
 'several days': 1,
 'more than half the days': 2,
 'nearly every day': 3}

In [13]:
OHIE['PHQ2_1'] = OHIE['dep_interest_12m'].replace(phq_scoring).astype('Int64')
OHIE['PHQ2_2'] = OHIE['dep_sad_12m'].replace(phq_scoring).astype('Int64')

<ipython-input-13-334a9c989cad>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  OHIE['PHQ2_1'] = OHIE['dep_interest_12m'].replace(phq_scoring).astype('Int64')
<ipython-input-13-334a9c989cad>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  OHIE['PHQ2_2'] = OHIE['dep_sad_12m'].replace(phq_scoring).astype('Int64')


In [15]:
OHIE[['dep_interest_12m','dep_sad_12m','PHQ2_1','PHQ2_2']].head()

dep_interest_12m              dep_sad_12m  PHQ2_1  PHQ2_2
0         nearly every day         nearly every day       3       3
1             several days             several days       1       1
2             several days               not at all       1       0
3  more than half the days         nearly every day       2       3
4  more than half the days  more than half the days       2       2

In [16]:
OHIE['PHQ2_sum'] = OHIE[['PHQ2_1','PHQ2_2']].sum(axis=1)

In [17]:
OHIE['PHQ2_cutoff'] = OHIE['PHQ2_sum'].apply(lambda x: 1 if x >= 3 else 0)

In [18]:
OHIE[['PHQ2_1','PHQ2_2','PHQ2_sum','PHQ2_cutoff']].head()

PHQ2_1  PHQ2_2  PHQ2_sum  PHQ2_cutoff
0       3       3         6            1
1       1       1         2            0
2       1       0         1            0
3       2       3         5            1
4       2       2         4            1

In [20]:
# How many people in this dataset had above-threashold depression symptoms?
OHIE['PHQ2_cutoff'].value_counts() / OHIE.shape[0] * 100

PHQ2_cutoff
0    67.75
1    32.25
Name: count, dtype: float64